# 📊 PA Break Strategy — Chart Viewer & Backtest

Notebook này giúp:
1. **Xem chart** candlestick tương tác (zoom, scroll)
2. **Hiển thị Swing Points** (HH, LL)
3. **Hiển thị tín hiệu** Entry / SL / TP
4. **Backtest** và xem kết quả

Bạn có thể nói với Copilot về đường giá mà Copilot sẽ thấy được data.

In [ ]:
# Cell 1: Import và setup
import sys
sys.path.insert(0, '.')

from fetch_data import fetch_ohlcv, show_available_symbols
from strategy_pa_break import run_pa_break, signals_to_dataframe, print_backtest_summary
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np

print("✅ Modules loaded!")
show_available_symbols()

In [ ]:
# Cell 2: ═══ CẤU HÌNH ═══  (Thay đổi tại đây)
SYMBOL    = "XAUUSD"    # Cặp tiền/hàng hóa
TIMEFRAME = "D1"        # M1, M5, M15, M30, H1, H4, D1, W1
BARS      = 300         # Số nến lấy
PIVOT_LEN = 5           # Pivot lookback
RR_RATIO  = 2.0         # Risk:Reward ratio

# Lấy data
df = fetch_ohlcv(SYMBOL, TIMEFRAME, bars=BARS)
print(f"\nData shape: {df.shape}")
df.tail()

In [ ]:
# Cell 3: Chạy Strategy
signals, swings = run_pa_break(df, pivot_len=PIVOT_LEN, rr_ratio=RR_RATIO)

print(f"🔍 Swing Points: {len(swings)}")
print(f"📈 Signals: {len(signals)}")

# Backtest summary
print_backtest_summary(signals, rr_ratio=RR_RATIO)

In [ ]:
# Cell 4: Vẽ Interactive Chart với Plotly
fig = make_subplots(rows=2, cols=1, shared_xaxes=True,
                    row_heights=[0.8, 0.2],
                    vertical_spacing=0.02)

# Candlestick
fig.add_trace(go.Candlestick(
    x=df.index, open=df['Open'], high=df['High'],
    low=df['Low'], close=df['Close'],
    name='Price',
    increasing_line_color='#26a69a',
    decreasing_line_color='#ef5350',
), row=1, col=1)

# Swing Highs
sh_points = [s for s in swings if s.type == 'HIGH']
if sh_points:
    fig.add_trace(go.Scatter(
        x=[s.time for s in sh_points],
        y=[s.price for s in sh_points],
        mode='markers', name='Swing High',
        marker=dict(symbol='triangle-down', size=10, color='orange'),
    ), row=1, col=1)

# Swing Lows
sl_points = [s for s in swings if s.type == 'LOW']
if sl_points:
    fig.add_trace(go.Scatter(
        x=[s.time for s in sl_points],
        y=[s.price for s in sl_points],
        mode='markers', name='Swing Low',
        marker=dict(symbol='triangle-up', size=10, color='cornflowerblue'),
    ), row=1, col=1)

# Buy Signals
buy_sigs = [s for s in signals if s.direction == 'BUY']
if buy_sigs:
    for sig in buy_sigs:
        color = '#26a69a' if sig.pnl_r > 0 else '#ef5350' if sig.result == 'SL' else '#888'
        # Entry line
        fig.add_shape(type='line',
            x0=sig.break_time, x1=sig.time,
            y0=sig.entry, y1=sig.entry,
            line=dict(color='dodgerblue', width=1, dash='dash'),
            row=1, col=1)
        # SL line
        fig.add_shape(type='line',
            x0=sig.break_time, x1=sig.time,
            y0=sig.sl, y1=sig.sl,
            line=dict(color='yellow', width=1, dash='dash'),
            row=1, col=1)
        # TP line
        if sig.tp > 0:
            fig.add_shape(type='line',
                x0=sig.break_time, x1=sig.time,
                y0=sig.tp, y1=sig.tp,
                line=dict(color='limegreen', width=1, dash='dash'),
                row=1, col=1)
    # Buy markers
    fig.add_trace(go.Scatter(
        x=[s.time for s in buy_sigs],
        y=[s.entry for s in buy_sigs],
        mode='markers+text', name='BUY',
        marker=dict(symbol='triangle-up', size=14, color='dodgerblue'),
        text=['▲BUY' for _ in buy_sigs],
        textposition='top center', textfont=dict(size=9),
    ), row=1, col=1)

# Sell Signals
sell_sigs = [s for s in signals if s.direction == 'SELL']
if sell_sigs:
    for sig in sell_sigs:
        # Entry line
        fig.add_shape(type='line',
            x0=sig.break_time, x1=sig.time,
            y0=sig.entry, y1=sig.entry,
            line=dict(color='hotpink', width=1, dash='dash'),
            row=1, col=1)
        # SL line
        fig.add_shape(type='line',
            x0=sig.break_time, x1=sig.time,
            y0=sig.sl, y1=sig.sl,
            line=dict(color='yellow', width=1, dash='dash'),
            row=1, col=1)
        # TP line
        if sig.tp > 0:
            fig.add_shape(type='line',
                x0=sig.break_time, x1=sig.time,
                y0=sig.tp, y1=sig.tp,
                line=dict(color='limegreen', width=1, dash='dash'),
                row=1, col=1)
    # Sell markers
    fig.add_trace(go.Scatter(
        x=[s.time for s in sell_sigs],
        y=[s.entry for s in sell_sigs],
        mode='markers+text', name='SELL',
        marker=dict(symbol='triangle-down', size=14, color='hotpink'),
        text=['▼SELL' for _ in sell_sigs],
        textposition='bottom center', textfont=dict(size=9),
    ), row=1, col=1)

# Volume
colors = ['#26a69a' if c >= o else '#ef5350' for o, c in zip(df['Open'], df['Close'])]
fig.add_trace(go.Bar(
    x=df.index, y=df['Volume'], name='Volume',
    marker_color=colors, opacity=0.5,
), row=2, col=1)

# Layout
fig.update_layout(
    title=f'{SYMBOL} {TIMEFRAME} — PA Break Strategy (RR 1:{RR_RATIO})',
    template='plotly_dark',
    height=800,
    xaxis_rangeslider_visible=False,
    showlegend=True,
    legend=dict(x=0, y=1.12, orientation='h'),
)
fig.update_yaxes(title_text='Price', row=1, col=1)
fig.update_yaxes(title_text='Volume', row=2, col=1)

fig.show()

In [ ]:
# Cell 5: Xem chi tiết các signals
sig_df = signals_to_dataframe(signals)
if not sig_df.empty:
    print(f"📋 Tất cả {len(sig_df)} signals:\n")
    # Hiển thị đẹp hơn
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', 200)
    display(sig_df)
else:
    print("Không có signal nào.")

In [ ]:
# Cell 6: So sánh nhiều RR ratios
print("📊 So sánh RR Ratios:\n")
rr_values = [1.0, 1.5, 2.0, 2.5, 3.0]
results = []

for rr in rr_values:
    sigs, _ = run_pa_break(df, pivot_len=PIVOT_LEN, rr_ratio=rr)
    sig_df = signals_to_dataframe(sigs)
    closed = sig_df[sig_df['Result'].isin(['TP', 'SL', 'CLOSE_REVERSE'])]
    if len(closed) > 0:
        wins = len(closed[closed['PnL (R)'] > 0])
        wr = wins / len(closed) * 100
        total_r = closed['PnL (R)'].sum()
        expectancy = wr/100 * rr - (1 - wr/100)
        results.append({
            'RR': f'1:{rr}',
            'Trades': len(closed),
            'Win Rate': f'{wr:.1f}%',
            'Total PnL (R)': f'{total_r:.2f}',
            'Avg PnL (R)': f'{closed["PnL (R)"].mean():.2f}',
            'Expectancy': f'{expectancy:.2f} R',
        })

if results:
    comparison = pd.DataFrame(results)
    display(comparison)
else:
    print("Không đủ data để so sánh.")

In [ ]:
# Cell 7: Equity Curve
if signals:
    sig_df = signals_to_dataframe(signals)
    closed = sig_df[sig_df['Result'].isin(['TP', 'SL', 'CLOSE_REVERSE'])].copy()
    
    if len(closed) > 0:
        closed['Cumulative R'] = closed['PnL (R)'].cumsum()
        
        fig_eq = go.Figure()
        fig_eq.add_trace(go.Scatter(
            x=list(range(1, len(closed)+1)),
            y=closed['Cumulative R'].values,
            mode='lines+markers',
            name='Equity (R)',
            line=dict(color='dodgerblue', width=2),
            marker=dict(
                size=8,
                color=['green' if r > 0 else 'red' for r in closed['PnL (R)']],
            ),
        ))
        fig_eq.add_hline(y=0, line_dash='dash', line_color='gray')
        fig_eq.update_layout(
            title=f'Equity Curve — {SYMBOL} {TIMEFRAME} (RR 1:{RR_RATIO})',
            template='plotly_dark',
            xaxis_title='Trade #',
            yaxis_title='Cumulative R',
            height=400,
        )
        fig_eq.show()
    else:
        print("Chưa có lệnh đóng.")
else:
    print("Không có signal.")

## 💡 Cách sử dụng với Copilot

Bây giờ bạn có thể hỏi Copilot:

1. **"Xem 100 nến EURUSD M15 gần nhất"** → Thay SYMBOL/TIMEFRAME rồi chạy lại
2. **"Tại sao signal BUY ở ngày X lại lỗ?"** → Copilot đọc sig_df
3. **"Thử thay pivot_len = 3 xem win rate thay đổi thế nào"**
4. **"Thêm filter: chỉ trade khi giá trên EMA 200"**
5. **"Convert logic này sang MQL5"** → Copilot đã thấy cả Python + MQL5 code

---
## 🔍 Công cụ khám phá giá — Dùng để mô tả cho Copilot

Các cell bên dưới giúp bạn:
1. **Xem giá quanh 1 ngày cụ thể** → "Ngày này giá tạo HH"
2. **Tìm swing points gần 1 ngày** → "Swing high tại đây là break point"
3. **Zoom chart vào 1 khoảng thời gian** → xem rõ cấu trúc giá

In [ ]:
# Cell 8: 🔍 Xem giá quanh 1 ngày cụ thể
# ═══ THAY ĐỔI NGÀY TẠI ĐÂY ═══
LOOKUP_DATE = "2025-10-01"   # Ngày bạn muốn xem
CONTEXT_BARS = 10            # Số nến trước và sau

# Tìm bars gần ngày đó
target = pd.Timestamp(LOOKUP_DATE)
idx = df.index.get_indexer([target], method='nearest')[0]
start = max(0, idx - CONTEXT_BARS)
end = min(len(df), idx + CONTEXT_BARS + 1)

context_df = df.iloc[start:end].copy()
context_df['Bar#'] = range(start, end)

# Highlight ngày target
print(f"📍 Xem giá quanh ngày {LOOKUP_DATE}")
print(f"   Bar gần nhất: {df.index[idx]} (index={idx})")
print(f"   Hiển thị {CONTEXT_BARS} nến trước/sau:\n")

# Format đẹp
pd.set_option('display.float_format', lambda x: f'{x:.5f}')
display(context_df)

# Thông tin swing points gần khu vực này
nearby_swings = [s for s in swings 
                 if df.index[start] <= s.time <= df.index[end-1]]
if nearby_swings:
    print(f"\n🔄 Swing points trong khu vực:")
    for s in nearby_swings:
        print(f"   {'🔺' if s.type=='HIGH' else '🔻'} {s.type}: "
              f"{s.price:.5f} @ {s.time.strftime('%Y-%m-%d')}")
else:
    print("\n   Không có swing point trong khu vực này.")

In [ ]:
# Cell 9: 📋 Danh sách TẤT CẢ Swing Points (để bạn chọn và mô tả)
print(f"📋 Tất cả {len(swings)} Swing Points:\n")
print(f"{'#':>3} | {'Type':>5} | {'Price':>12} | {'Date':>12} | {'Bar#':>5}")
print("-" * 55)

for i, s in enumerate(swings):
    marker = "🔺" if s.type == "HIGH" else "🔻"
    print(f"{i:3d} | {marker} {s.type:>5} | {s.price:12.5f} | "
          f"{s.time.strftime('%Y-%m-%d'):>12} | {s.bar_index:5d}")

print(f"\n💡 Tip: Copy dòng cụ thể rồi nói với Copilot:")
print(f'   "Swing High #12 tại giá 3050 ngày 2025-03-19 là break point."')
print(f'   "Entry tại Swing High #10 (2957), SL tại Swing Low #11 (2828)"')

In [ ]:
# Cell 10: 🔎 Zoom Chart vào khoảng thời gian cụ thể
# ═══ THAY ĐỔI NGÀY TẠI ĐÂY ═══
ZOOM_START = "2025-09-01"
ZOOM_END   = "2025-12-31"

zoom_df = df.loc[ZOOM_START:ZOOM_END]
zoom_swings = [s for s in swings if pd.Timestamp(ZOOM_START) <= s.time <= pd.Timestamp(ZOOM_END)]
zoom_signals = [s for s in signals if pd.Timestamp(ZOOM_START) <= s.time <= pd.Timestamp(ZOOM_END)]

fig_zoom = go.Figure()

# Candlestick
fig_zoom.add_trace(go.Candlestick(
    x=zoom_df.index, open=zoom_df['Open'], high=zoom_df['High'],
    low=zoom_df['Low'], close=zoom_df['Close'],
    name='Price',
    increasing_line_color='#26a69a',
    decreasing_line_color='#ef5350',
))

# Swing points
for s in zoom_swings:
    color = 'orange' if s.type == 'HIGH' else 'cornflowerblue'
    symbol = 'triangle-down' if s.type == 'HIGH' else 'triangle-up'
    fig_zoom.add_trace(go.Scatter(
        x=[s.time], y=[s.price],
        mode='markers+text', showlegend=False,
        marker=dict(symbol=symbol, size=12, color=color),
        text=[f"{s.price:.2f}"],
        textposition='top center' if s.type == 'HIGH' else 'bottom center',
        textfont=dict(size=9, color=color),
    ))

# Signals
for sig in zoom_signals:
    entry_color = 'dodgerblue' if sig.direction == 'BUY' else 'hotpink'
    # Entry
    fig_zoom.add_shape(type='line',
        x0=sig.break_time, x1=sig.time,
        y0=sig.entry, y1=sig.entry,
        line=dict(color=entry_color, width=2, dash='dash'))
    # SL
    fig_zoom.add_shape(type='line',
        x0=sig.break_time, x1=sig.time,
        y0=sig.sl, y1=sig.sl,
        line=dict(color='yellow', width=1, dash='dash'))
    # TP
    if sig.tp > 0:
        fig_zoom.add_shape(type='line',
            x0=sig.break_time, x1=sig.time,
            y0=sig.tp, y1=sig.tp,
            line=dict(color='limegreen', width=1, dash='dash'))
    # Label
    fig_zoom.add_annotation(
        x=sig.time, y=sig.entry,
        text=f"{'▲BUY' if sig.direction=='BUY' else '▼SELL'} | {sig.result}",
        showarrow=True, arrowhead=2,
        font=dict(size=10, color=entry_color),
    )

fig_zoom.update_layout(
    title=f'{SYMBOL} {TIMEFRAME} | Zoom: {ZOOM_START} → {ZOOM_END}',
    template='plotly_dark',
    height=600,
    xaxis_rangeslider_visible=False,
)
fig_zoom.show()

print(f"\n📊 Khu vực {ZOOM_START} → {ZOOM_END}:")
print(f"   Nến: {len(zoom_df)} | Swings: {len(zoom_swings)} | Signals: {len(zoom_signals)}")

## 🗣️ Ví dụ cách nói với Copilot

Sau khi chạy notebook, bạn đã có data. Giờ mở Copilot Chat và nói:

**Ví dụ 1 — Mô tả entry:**
> "Nhìn vào file pa_break_analysis.ipynb, ngày 2025-03-19 có Swing High = 3050 tạo HH (break up). 
> Entry tại swing cũ 2957, SL tại swing low 2828. Hãy viết logic này thành Pine Script."

**Ví dụ 2 — Hỏi về TP:**
> "Với data XAUUSD D1, pivot_len=5, RR=2.0, win rate đang 100% nhưng chỉ có 4 signals. 
> Nếu tôi giảm pivot_len=3, signals sẽ nhiều hơn nhưng win rate có giảm không? Thử xem."

**Ví dụ 3 — Thêm filter:**
> "Tôi thấy trong backtest, signal BUY ngày 2025-04-29 entry tại 3168 thắng đẹp. 
> Nhưng tôi muốn thêm filter: chỉ BUY khi giá trên EMA 50. Hãy thêm vào strategy_pa_break.py."

**Ví dụ 4 — Convert sang MQL5:**
> "Logic trong strategy_pa_break.py giống Expert PA Break.mq5. 
> Hãy thêm EMA filter vào Expert PA Break.mq5 giống như trong Python."

**Key point:** Copilot đọc được cả notebook output + Python files + MQL5 files trong workspace.